<center> <h2>Universidad de Sonora </h2></center>
<center> <h2>Ariel David López Cota </h2></center>
<center> <h2>Estadistica MCD </h2></center>
<!-- <center> <h2>  </h2></center> -->
<center> 

![](unison.png)

</center>






<center> Introducción </center>
<p> Esta presentación es para un caso de estudio de biociencia de Bella Anahi, aqui uncluire codigo asi como explicación de lo que realize.</p>
<p> Me enfoqué en la realización de una visualización de a traves del tiempo, apliqué 3 metodos de cluster, espero sean de utilidad.</p>

In [265]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

#### Lectura de los datos
Base de datos proporcionados por Bella Flor

In [2]:
# Read Excel file
df_TB4 = pd.read_excel("TB4.xlsx")

Creacion de una tabla, con 3 columnas, las especies/familia/genero estaran en una columna

In [67]:
# We will create a new table categorizing the data, to reduce the columns convetirng them to a "Especie" column
df_TB4_Test = df_TB4.copy()
df_Ordered = pd.DataFrame(columns=["Profundidad", "Element", "Valor"])

list_columns = df_TB4_Test.columns[df_TB4_Test.columns != "Profundidad"]

for col in list_columns:
    for nrow in range(0,len(df_TB4_Test.index)):
        specie = col
        profundidad = df_TB4_Test["Profundidad"].iloc[nrow]
        valor = df_TB4_Test[col].iloc[nrow]
        # Add a new row
        df_Ordered.loc[len(df_Ordered.index)] = [profundidad, specie, valor] 

df_Ordered

,Profundidad,Element,Valor
0,5.0,Abies,0.000
1,10.0,Abies,0.000
2,15.0,Abies,0.000
3,18.0,Abies,0.000
4,21.0,Abies,0.000
...,...,...,...
12982,322.5,Indeterminables,3.125
12983,325.0,Indeterminables,3.667
12984,327.5,Indeterminables,1.242
12985,330.0,Indeterminables,0.000


In [99]:
# CLEAN THIS VALUES
# len(df_Ordered.Profundidad.unique())
# len(df_Ordered.Element.unique())
# len(df_clasification_clean.Element.unique())
# df_TB4
# df_Ordered
# df_Ordered.Element.unique()
# df_clasification_clean.Element.unique()
list_dif = list(set(df_Ordered.Element.unique()) - set(df_clasification_clean.Element.unique()))
list_dif

['Indeterminables',
 'Asplenium ',
 'Euphorbia ',
 'Indeterminados',
 'Poaceae >60µm',
 'Tribulus ',
 'Kallstroemia ',
 'Unknown trilete']

## Line charts

Con base a los datos ordenados, ahora somos capaces de crear un grafico que nos permita visualizar de manera clara y sencilla, la proporcion que exsitia atraves del tiempo

In [158]:

fig = go.Figure()

for specie in df_Ordered["Especie"].unique():
    if specie == "Pinus":
        fig.add_trace(go.Scatter(x=df_Ordered[df_Ordered["Especie"] == specie]["Profundidad"], y=df_Ordered[df_Ordered["Especie"] == specie]["Valor"], name=specie,
                         line = dict(color='green', width=4)))
    else:
        fig.add_trace(go.Scatter(x=df_Ordered[df_Ordered["Especie"] == specie]["Profundidad"], y=df_Ordered[df_Ordered["Especie"] == specie]["Valor"], name=specie,
                         line = dict(color='gray', width=1)))

fig.update_layout(title='Cantidad de proporcionn',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()


##
Dejamos esta grafico para que el lector pueda realizar comparaciones 

In [60]:
import plotly.graph_objects as go
fig = go.Figure()

for specie in df_Ordered["Especie"].unique():
    if specie == "Pinus":
        fig.add_trace(go.Scatter(x=df_Ordered[df_Ordered["Especie"] == specie]["Profundidad"], y=df_Ordered[df_Ordered["Especie"] == specie]["Valor"], name=specie,
                         line = dict(color='green', width=4)))
    elif specie == "Asteroideae":
        fig.add_trace(go.Scatter(x=df_Ordered[df_Ordered["Especie"] == specie]["Profundidad"], y=df_Ordered[df_Ordered["Especie"] == specie]["Valor"], name=specie,
                         line = dict(color='red', width=4)))
    else:
        fig.add_trace(go.Scatter(x=df_Ordered[df_Ordered["Especie"] == specie]["Profundidad"], y=df_Ordered[df_Ordered["Especie"] == specie]["Valor"], name=specie,
                         line = dict(color='gray', width=1)))

fig.update_layout(title='Cantidad de proporcionn',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()

<p>En base a los datos y sus propiedades he decidido realizar clusters, para una visualización mas clara con menos objetos en el grafico.</p>
<p>¿Que criterios se tomaran para definir los clusters?</p>
<p>Se segregaran por Genero y Familia, con la ayuda</p>

---

### Preambulo
Una de las problematicas del problema planteado por nuestro "Cliente", es que los elemntos que proporciona como una propiedad (un columna por cada dato), no son homologos uno del otro, por lo cual considero que no es correcto hacer comparaciones uno con el otro. 

<h4>¿Cual es problema con estas propiedades?</h4>
<p>El cliente nos comenta que a veces nos posible identificar la especie del polen, pero si su genero o su familia, entonces tenemos 3 tipos de elementos, los cuales si no hacemos una segregación (opinion personal) podriamos mostrar resultados sesgados y/o erroneos.</p>
<p>Para explicar mejor esta problematica abajo expongo un ejemplo en una imagen, como pueden ver observar la especie "Cardinalis cardinales" se encuentra en todas las "etapas" de la taxonomia, es decir la familia Fringilidae contiene a Cardinalis cardinales asi como tambien el genero Cardinales lo contiene. Entonces tomando esto en cuenta, pudieramos encontrarnos que una especie no este presente o en bastante proprcion con lo datos, pero realmente si pudiera estar bastante presente poruqe tal vez este incluido en su genero o su familia.</p>

![](taxonomia-animal.jpg)

https://www.areaciencias.com/biologia/taxonomia-clasificacion-de-los-seres-vivos/

<h4>¿Como pudieramos resolver este problematica o realizar una representación mas precisa?</h4>

Para este caso particualar, la solucion la he dividido en 4 pasos:
- Clasificación de los elementos
   - Los elementos seran clasificados por su tipo (Especie, Genero y Familia)
- Limpieza y carga de la clasificación
- Creación de clusters
- Visualización de clusters
---

<h3> 1 - Clasificación de los elementos </h3>

Debido al poco tiempo, la cantidad de datos y mi escazo conocimiento del tema, he recurrido a una herramienta de Inteligencia Artificial para que me realice esta clasificación de manera rapida y eficaz.
Como podran ver en la imagen, he logrado que me clasifique los elementos en una tabla, para ello tuve que realizar varias preguntas y pulirlas para lograr el resultado deseado.

![](ChatGPT_Classification.png)

<h4>¿Esta clasificación es correcta?</h4>
<p>Existe una alta probabilidad que todos resultados no sean correctos, pero, claro esta que dejamos al lector los datos, para que si desea una información mas precisa, pueda modificarlos.</p>

---

<h3> 2 - Limpieza y Carga de la clasificación </h3>
<p>Ya con el resultado en una tabla procedere a realizar una limpieza de los datos, habiendo extraido la tabla y pasandolo a un archivo csv.</p>

In [32]:
# Limpieza de los datos
df_clasification = pd.read_csv('Clasification.csv', encoding = "utf-8")
df_clasification

,Element,Type,Species,Genus,Family
0,Abies,Species,Abies,Abies,Pinaceae
1,Acacia,Species,Acacia,Acacia,Fabaceae
2,Agave,Species,Agave,Agave,Asparagaceae
3,Alnus oblongifolia,Species,Alnus oblongifolia,Alnus,Betulaceae
4,Anisacanthus type,Species,Anisacanthus,Anisacanthus,Acanthaceae
...,...,...,...,...,...
112,Pellaea,Genus,-,Pellaea,Pteridaceae
113,Spore trilete without perine,Species,-,Unknown,Unknown
114,Unknown trilete,Species,-,Unknown,Unknown
115,Indeterminados,Species,-,Unknown,Unknown


In [52]:
# Familia desconocida, elementos no parecen ser ninguno
df_clasification[(df_clasification['Species'] == '-') & (df_clasification['Type'] == 'Species')]

,Element,Type,Species,Genus,Family
114,Unknown trilete,Species,-,Unknown,Unknown
115,Indeterminados,Species,-,Unknown,Unknown
116,Indeterminables,Species,-,Unknown,Unknown


In [53]:
# Corregiremos 4 valores (Cf.Populus, Reticulate trilete, )
df_clasification.loc[14].Species = df_clasification.loc[14].Element
df_clasification.loc[81].Type = 'Family'
df_clasification.loc[110].Species = df_clasification.loc[110].Element
df_clasification.loc[113].Species = df_clasification.loc[113].Element
# Quitaremos los elementos desconocidos, no creo que sea de utilidad conocer la proporcion de elementos desconocidos (a reserva que le lector lo desea, puede dejarlos)
df_clasification_clean = df_clasification[df_clasification.index < 114].copy()
df_clasification_clean

,Element,Type,Species,Genus,Family
0,Abies,Species,Abies,Abies,Pinaceae
1,Acacia,Species,Acacia,Acacia,Fabaceae
2,Agave,Species,Agave,Agave,Asparagaceae
3,Alnus oblongifolia,Species,Alnus oblongifolia,Alnus,Betulaceae
4,Anisacanthus type,Species,Anisacanthus,Anisacanthus,Acanthaceae
...,...,...,...,...,...
109,Osmunda,Genus,-,Osmunda,Osmundaceae
110,Reticulate trilete,Species,Reticulate trilete,Unknown,Unknown
111,Selaginella,Genus,-,Selaginella,Selaginellaceae
112,Pellaea,Genus,-,Pellaea,Pteridaceae


A continuación crearemos un nuevo dataframe (tabla) para almacenar los valores por cada profundiad por familia, siendo la suma de los valores de todas las especies y generos que pertenezcan a la familia.

In [111]:
df_Ordered_Complete = df_Ordered.merge(df_clasification_clean, on='Element')
df_Ordered_Complete

# Creación de un nuevo dataframe para albergar solo las familias
df_elements_by_famliy = pd.DataFrame(columns=["Profundidad", "Family", "Valor"])
for fam in df_Ordered_Complete["Family"].unique():
    for prof in df_Ordered_Complete["Profundidad"].unique():
        sumFamElements = sum(df_Ordered_Complete[(df_Ordered_Complete["Family"] == fam) & (df_Ordered_Complete["Profundidad"] == prof)]["Valor"])
        df_elements_by_famliy.loc[len(df_elements_by_famliy.index)] = [prof, fam, sumFamElements] 


#### Creación de ..

In [231]:
# Creación de tabla cada familia una columna (Propiedad)
df_features_1 = pd.DataFrame()
# df_features_1.insert(0, "Profundidad", df_elements_by_famliy["Profundidad"].unique(), True)
countCol = 0
for fam in df_elements_by_famliy["Family"].unique():
    df_features_1.insert(countCol, fam, list(df_elements_by_famliy[df_elements_by_famliy["Family"] == fam]["Valor"]), True)
    countCol = countCol + 1
df_features_1.insert(0, 'Profundidad', list(df_elements_by_famliy["Profundidad"].unique()), True)
df_features_1

,Profundidad,Pinaceae,Fabaceae,Asparagaceae,Betulaceae,Acanthaceae,Ericaceae,Berberidaceae,Burseraceae,Cactaceae,...,Zygophyllaceae,Pteridaceae,Anemiaceae,Ophioglossaceae,Dryopteridaceae,Lygodiaceae,Lycopodiaceae,Osmundaceae,Unknown,Selaginellaceae
0,5.0,5.508,15.254,0.0,0.000,0.000,0.0,0.0,0.0,2.541,...,0.0,0.424,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.424
1,10.0,3.665,20.943,0.0,0.000,0.000,0.0,0.0,0.0,4.712,...,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.524
2,15.0,3.371,11.798,0.0,0.000,0.562,0.0,0.0,0.0,5.618,...,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.000,1.124
3,18.0,0.000,1.667,0.0,0.000,0.000,0.0,0.0,0.0,0.000,...,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.000
4,21.0,1.724,0.000,0.0,0.000,0.000,0.0,0.0,0.0,6.896,...,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,322.5,53.693,0.284,0.0,0.568,0.000,0.0,0.0,0.0,0.000,...,0.0,0.568,0.0,0.000,0.0,0.000,0.0,0.000,1.988,0.852
107,325.0,52.000,0.000,0.0,0.667,0.000,0.0,0.0,0.0,0.000,...,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,1.333,0.000
108,327.5,61.181,0.000,0.0,0.311,0.000,0.0,0.0,0.0,0.000,...,0.0,0.000,0.0,0.311,0.0,0.000,0.0,0.000,0.000,1.242
109,330.0,67.481,0.000,0.0,0.244,0.000,0.0,0.0,0.0,0.000,...,0.0,0.000,0.0,0.000,0.0,1.711,0.0,0.000,0.244,0.000


---

### Method 1 -  Agglomerative Clustering 

1 - Creation a numeric variables for cluster clasification
2 - Create cluster

In [254]:
df_features_1_15 = df_elements_by_famliy.copy()
df_features_1_15["Family_Numeric"] = pd.factorize(df_features_1_15["Family"])[0]
df_features_1_15 = df_features_1_15.drop(columns='Family')
# Instantiate the AgglomerativeClustering model with the desired number of clusters
agg_clustering = AgglomerativeClustering(n_clusters=15)

# Fit the model to the data
agg_clustering.fit(df_features_1_15)

# Add the cluster labels to the dataframe
df_features_1_15['Cluster'] = agg_clustering.labels_
df_features_1_15["Family"] = df_elements_by_famliy["Family"]
#------------------------------------
df_features_1_31 = df_elements_by_famliy.copy()
df_features_1_31["Family_Numeric"] = pd.factorize(df_features_1_31["Family"])[0]
df_features_1_31 = df_features_1_31.drop(columns='Family')
# Instantiate the AgglomerativeClustering model with the desired number of clusters
agg_clustering = AgglomerativeClustering(n_clusters=31)

# Fit the model to the data
agg_clustering.fit(df_features_1_31)

# Add the cluster labels to the dataframe
df_features_1_31['Cluster'] = agg_clustering.labels_
df_features_1_31["Family"] = df_elements_by_famliy["Family"]
# -------------------------------------------------
df_features_1_62 = df_elements_by_famliy.copy()
df_features_1_62["Family_Numeric"] = pd.factorize(df_features_1_62["Family"])[0]
df_features_1_62 = df_features_1_62.drop(columns='Family')
# Instantiate the AgglomerativeClustering model with the desired number of clusters
agg_clustering = AgglomerativeClustering(n_clusters=62)

# Fit the model to the data
agg_clustering.fit(df_features_1_62)

# Add the cluster labels to the dataframe
df_features_1_62['Cluster'] = agg_clustering.labels_
df_features_1_62["Family"] = df_elements_by_famliy["Family"]

### Method 2 - Kmeans

In [260]:
df_features_2_15 = df_elements_by_famliy.copy()
df_features_2_15["Family_Numeric"] = pd.factorize(df_features_2_15["Family"])[0]
df_features_2_15 = df_features_2_15.drop(columns='Family')
# Instantiate the KMeans model with the desired number of clusters
kmeans = KMeans(n_clusters=15)

# Fit the model to the data
kmeans.fit(df_features_2_15)

# Add the cluster labels to the dataframe
df_features_2_15['Cluster'] = kmeans.labels_
df_features_2_15["Family"] = df_elements_by_famliy["Family"]
#------------------------------------------------
df_features_2_31 = df_elements_by_famliy.copy()
df_features_2_31["Family_Numeric"] = pd.factorize(df_features_2_31["Family"])[0]
df_features_2_31 = df_features_2_31.drop(columns='Family')
# Instantiate the KMeans model with the desired number of clusters
kmeans = KMeans(n_clusters=15)

# Fit the model to the data
kmeans.fit(df_features_2_31)

# Add the cluster labels to the dataframe
df_features_2_31['Cluster'] = kmeans.labels_
df_features_2_31["Family"] = df_elements_by_famliy["Family"]
#------------------------------------------------
df_features_2_62 = df_elements_by_famliy.copy()
df_features_2_62["Family_Numeric"] = pd.factorize(df_features_2_62["Family"])[0]
df_features_2_62 = df_features_2_62.drop(columns='Family')
# Instantiate the KMeans model with the desired number of clusters
kmeans = KMeans(n_clusters=62)

# Fit the model to the data
kmeans.fit(df_features_2_62)

# Add the cluster labels to the dataframe
df_features_2_62['Cluster'] = kmeans.labels_
df_features_2_62["Family"] = df_elements_by_famliy["Family"]

#### Method 3 - DBSCAN
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

In [280]:
df_features_3_15 = df_elements_by_famliy.copy()
df_features_3_15["Family_Numeric"] = pd.factorize(df_features_3_15["Family"])[0]
df_features_3_15 = df_features_3_15.drop(columns='Family')
# Instantiate the DBSCAN model with the desired parameters
dbscan = DBSCAN(eps=4, min_samples=3)

# Fit the model to the data
dbscan.fit(df_features_3_15)

# Add the cluster labels to the dataframe
df_features_3_15['Cluster'] = dbscan.labels_
df_features_3_15["Family"] = df_elements_by_famliy["Family"]
# --------------------------------------------------------------
df_features_3_31 = df_elements_by_famliy.copy()
df_features_3_31["Family_Numeric"] = pd.factorize(df_features_3_31["Family"])[0]
df_features_3_31 = df_features_3_31.drop(columns='Family')
# Instantiate the DBSCAN model with the desired parameters
dbscan = DBSCAN(eps=0.5, min_samples=31)

# Fit the model to the data
dbscan.fit(df_features_3_31)

# Add the cluster labels to the dataframe
df_features_3_31['Cluster'] = dbscan.labels_
df_features_3_31["Family"] = df_elements_by_famliy["Family"]
# --------------------------------------------------------------
df_features_3_62 = df_elements_by_famliy.copy()
df_features_3_62["Family_Numeric"] = pd.factorize(df_features_3_62["Family"])[0]
df_features_3_62 = df_features_3_62.drop(columns='Family')
# Instantiate the DBSCAN model with the desired parameters
dbscan = DBSCAN(eps=0.5, min_samples=62)

# Fit the model to the data
dbscan.fit(df_features_3_62)

# Add the cluster labels to the dataframe
df_features_3_62['Cluster'] = dbscan.labels_
df_features_3_62["Family"] = df_elements_by_famliy["Family"]

In [270]:
df_elements_by_famliy

,Profundidad,Family,Valor
0,5.0,Pinaceae,5.508
1,10.0,Pinaceae,3.665
2,15.0,Pinaceae,3.371
3,18.0,Pinaceae,0.000
4,21.0,Pinaceae,1.724
...,...,...,...
6877,322.5,Selaginellaceae,0.852
6878,325.0,Selaginellaceae,0.000
6879,327.5,Selaginellaceae,1.242
6880,330.0,Selaginellaceae,0.000


###  ¿Cual es la diferencia ente ambos?
https://medium.com/@namanbhandari/agglomerative-clustering-vs-k-means-clustering-766a90b37dc0

In [234]:
 # We will create a new table categorizing the data, to reduce the columns convetirng them to a "Family" column
# df_features_1.insert(0, "Profundidad", df_elements_by_famliy["Profundidad"].unique(), True)

df_family_Clustered_1 = pd.DataFrame(columns=["Profundidad", "Family", "Valor", "Cluster"])

list_columns = df_features_1.columns[(df_features_1.columns != "Profundidad") & (df_features_1.columns != "Cluster")]

for col in list_columns:
    for nrow in range(0,len(df_features_1.index)):
        family = col
        profundidad = df_features_1["Profundidad"].iloc[nrow]
        valor = df_features_1[col].iloc[nrow]
        cluster = df_features_1["Cluster"].iloc[nrow]
        # Add a new row
        df_family_Clustered_1.loc[len(df_family_Clustered_1.index)] = [profundidad, family, valor, cluster] 

df_family_Clustered_1

,Profundidad,Family,Valor,Cluster
0,5.0,Pinaceae,5.508,13
1,10.0,Pinaceae,3.665,13
2,15.0,Pinaceae,3.371,13
3,18.0,Pinaceae,0.000,16
4,21.0,Pinaceae,1.724,8
...,...,...,...,...
6877,322.5,Selaginellaceae,0.852,2
6878,325.0,Selaginellaceae,0.000,2
6879,327.5,Selaginellaceae,1.242,2
6880,330.0,Selaginellaceae,0.000,2


### Visualización
---

Method 1 - Cluster

In [257]:
df_features_1_31[df_features_1_31["Cluster"] == 27]["Family"].unique()

array(['Fagaceae', 'Amaranthaceae', 'Asteraceae', 'Poaceae'], dtype=object)

In [247]:
# df_family_Clustered_1
fig = go.Figure()

for element in df_features_1_15["Cluster"].unique():
    # fig.add_trace(go.Scatter(x=df_features_1[df_features_1["Cluster"] == element]["Profundidad"], y=df_features_1[df_features_1["Cluster"] == element]["Valor"], name=str(element)))
    fig.add_trace(go.Scatter(x=df_features_1_15[df_features_1_15["Cluster"] == element]["Profundidad"], y=df_features_1_15[df_features_1_15["Cluster"] == element]["Valor"], name=str(element)))
    # fig.add_trace(go.Scatter(x=df_family_Clustered_1[df_family_Clustered_1["Cluster"] == element]["Profundidad"], y=df_family_Clustered_1[df_family_Clustered_1["Cluster"] == element]["Valor"], name=str(element)))
    # if element == "Pinus":
    #     fig.add_trace(go.Scatter(x=df_elements_by_famliy[df_elements_by_famliy["Family"] == element]["Profundidad"], y=df_elements_by_famliy[df_elements_by_famliy["Family"] == element]["Valor"], name=element,
    #                      line = dict(color='green', width=4)))
    # else:
    #     fig.add_trace(go.Scatter(x=df_elements_by_famliy[df_elements_by_famliy["Family"] == element]["Profundidad"], y=df_elements_by_famliy[df_elements_by_famliy["Family"] == element]["Valor"], name=element,
    #                      line = dict(color='gray', width=1)))

fig.update_layout(title='Cantidad de proporcion',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()

In [258]:
fig = go.Figure()

for element in df_features_1_31["Cluster"].unique():
    nameCluster = str(element) + " (" +  str(len(df_features_1_31[df_features_1_31["Cluster"] == element]["Family"].unique())) + ")"
    fig.add_trace(go.Scatter(x=df_features_1_31[df_features_1_31["Cluster"] == element]["Profundidad"], y=df_features_1_31[df_features_1_31["Cluster"] == element]["Valor"], name=nameCluster))

fig.update_layout(title='Cantidad de proporcion',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()

In [259]:
fig = go.Figure()

for element in df_features_1_62["Cluster"].unique():
    nameCluster = str(element) + " (" +  str(len(df_features_1_62[df_features_1_62["Cluster"] == element]["Family"].unique())) + ")"
    fig.add_trace(go.Scatter(x=df_features_1_62[df_features_1_62["Cluster"] == element]["Profundidad"], y=df_features_1_62[df_features_1_62["Cluster"] == element]["Valor"], name=nameCluster))

fig.update_layout(title='Cantidad de proporcion',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()

#### Method 2 - KMeans

In [261]:
fig = go.Figure()

for element in df_features_2_15["Cluster"].unique():
    nameCluster = str(element) + " (" +  str(len(df_features_2_15[df_features_2_15["Cluster"] == element]["Family"].unique())) + ")"
    fig.add_trace(go.Scatter(x=df_features_2_15[df_features_2_15["Cluster"] == element]["Profundidad"], y=df_features_2_15[df_features_2_15["Cluster"] == element]["Valor"], name=nameCluster))

fig.update_layout(title='Cantidad de proporcion',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()

In [263]:
fig = go.Figure()

for element in df_features_2_31["Cluster"].unique():
    nameCluster = str(element) + " (" +  str(len(df_features_2_31[df_features_2_31["Cluster"] == element]["Family"].unique())) + ")"
    fig.add_trace(go.Scatter(x=df_features_2_31[df_features_2_31["Cluster"] == element]["Profundidad"], y=df_features_2_31[df_features_2_31["Cluster"] == element]["Valor"], name=nameCluster))

fig.update_layout(title='Cantidad de proporcion',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()

In [262]:
fig = go.Figure()

for element in df_features_2_62["Cluster"].unique():
    nameCluster = str(element) + " (" +  str(len(df_features_2_62[df_features_2_62["Cluster"] == element]["Family"].unique())) + ")"
    fig.add_trace(go.Scatter(x=df_features_2_62[df_features_2_62["Cluster"] == element]["Profundidad"], y=df_features_2_62[df_features_2_62["Cluster"] == element]["Valor"], name=nameCluster))

fig.update_layout(title='Cantidad de proporcion',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()

#### Method 3 - DBSCAN

In [281]:
fig = go.Figure()

for element in df_features_3_15["Cluster"].unique():
    nameCluster = str(element) + " (" +  str(len(df_features_3_15[df_features_3_15["Cluster"] == element]["Family"].unique())) + ")"
    fig.add_trace(go.Scatter(x=df_features_3_15[df_features_3_15["Cluster"] == element]["Profundidad"], y=df_features_3_15[df_features_3_15["Cluster"] == element]["Valor"], name=nameCluster))

fig.update_layout(title='Cantidad de proporcion',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()

In [ ]:
fig = go.Figure()

for element in df_features_2_62["Cluster"].unique():
    nameCluster = str(element) + " (" +  str(len(df_features_2_62[df_features_2_62["Cluster"] == element]["Family"].unique())) + ")"
    fig.add_trace(go.Scatter(x=df_features_2_62[df_features_2_62["Cluster"] == element]["Profundidad"], y=df_features_2_62[df_features_2_62["Cluster"] == element]["Valor"], name=nameCluster))

fig.update_layout(title='Cantidad de proporcion',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()

In [ ]:
fig = go.Figure()

for element in df_features_3_62["Cluster"].unique():
    nameCluster = str(element) + " (" +  str(len(df_features_2_62[df_features_2_62["Cluster"] == element]["Family"].unique())) + ")"
    fig.add_trace(go.Scatter(x=df_features_2_62[df_features_2_62["Cluster"] == element]["Profundidad"], y=df_features_2_62[df_features_2_62["Cluster"] == element]["Valor"], name=nameCluster))

fig.update_layout(title='Cantidad de proporcion',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()

In [113]:
# df_elements_by_famliy
fig = go.Figure()

for element in df_elements_by_famliy["Family"].unique():
    if element == "Pinus":
        fig.add_trace(go.Scatter(x=df_elements_by_famliy[df_elements_by_famliy["Family"] == element]["Profundidad"], y=df_elements_by_famliy[df_elements_by_famliy["Family"] == element]["Valor"], name=element,
                         line = dict(color='green', width=4)))
    else:
        fig.add_trace(go.Scatter(x=df_elements_by_famliy[df_elements_by_famliy["Family"] == element]["Profundidad"], y=df_elements_by_famliy[df_elements_by_famliy["Family"] == element]["Valor"], name=element,
                         line = dict(color='gray', width=1)))

fig.update_layout(title='Cantidad de proporcion',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()